# PyBrain

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2019-01-01'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,3.85,3.98,3.84,3.95,3.95,20548400
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700


In [3]:
X = dataset.iloc[ : , 1].values
y = dataset.iloc[ : , 4].values

In [4]:
print(X.shape)
print(y.shape)

(1258,)
(1258,)


In [5]:
X = np.array(X).reshape(1258,-1)
y = np.array(y).reshape(1258,-1)

In [6]:
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure import *
from pybrain.datasets import ClassificationDataSet
from pybrain.supervised.trainers import BackpropTrainer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

In [7]:
x = preprocessing.StandardScaler().fit_transform(X)
dataset = ClassificationDataSet(x.shape[1], 1, nb_classes=3)
for i in range(len(y)):
    dataset.addSample(list(x[i]), list([y[i]]))

In [8]:
dataTrain, dataTest = dataset.splitWithProportion(proportion=0.7)
x_train, y_train = dataTrain['input'], dataTrain['target']
x_test, y_test = dataTest['input'], dataTest['target']
print('Input dim:{} Output dim:{}'.format(dataTrain.indim, dataTrain.outdim))
print('Train: x = {} y = {}'.format(x_train.shape, y_train.shape))
print('Test: x = {} y = {}'.format(x_test.shape, y_test.shape))

Input dim:1 Output dim:1
Train: x = (880, 1) y = (880, 1)
Test: x = (378, 1) y = (378, 1)


In [9]:
net = buildNetwork(dataTrain.indim, 5, dataTrain.outdim, outclass=SoftmaxLayer)
model = BackpropTrainer(net, dataTrain, learningrate=0.01, momentum=0.1, verbose=False)
model.trainUntilConvergence(maxEpochs=100)

([], [42.970454545454544])

In [10]:
predict_train = np.argmax(net.activateOnDataset(dataTrain), axis=1)
actual_train = np.argmax(y_train, axis=1)
train_acc = accuracy_score(actual_train, predict_train)

In [11]:
predict_test = np.argmax(net.activateOnDataset(dataTest), axis=1)
actual_test = np.argmax(y_test, axis=1)
test_acc = accuracy_score(actual_test, predict_test)

In [12]:
print('Train acc = ', round(train_acc, 4), ' Test acc = ', round(test_acc, 4))

Train acc =  1.0  Test acc =  1.0


In [13]:
model = BackpropTrainer(net,dataTrain,learningrate=0.01,lrdecay=1.0,verbose=False)
model.trainUntilConvergence(maxEpochs=100)

predict = []
for arr in x_train:
    result = int(np.ceil(net.activate(arr)[0]))
    predict.append(result)
predict = np.array(predict)
print('Train acc = ',str(round(accuracy_score(y_train,predict)*100,2))+'%')

Train acc =  6.82%


In [14]:
predict = []
for arr in x_test:
    result = int(np.ceil(net.activate(arr)[0]))
    predict.append(result)
predict = np.array(predict)
print('Test acc = ',str(round(accuracy_score(y_test,predict)*100,2))+'%') 

Test acc =  5.56%
